#### Read in the Reddit Data

In [ ]:
import pickle
import sys
import os

parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(parent_directory)


# Define the path to the pickle file relative to the parent directory
pickle_file_path_comment = os.path.join(parent_directory, 'data_collection/project_data/climateCommentsDf.pickle')
pickle_file_path_submission = os.path.join(parent_directory, 'data_collection/project_data/climateSubmissionsDf.pickle')

print(pickle_file_path_comment)
# Now you can access the pickle file
with open(pickle_file_path_comment, 'rb') as f:
    comments_df = pickle.load(f)

print(pickle_file_path_submission)
# Now you can access the pickle file
with open(pickle_file_path_submission, 'rb') as f:
    submission_df = pickle.load(f)

In [ ]:
comments_df.head()
#subset_comments_df = comments_df.iloc[0:1000] for testing
#subset_comments_df.head()

In [ ]:
comments_df.to_csv("comments.csv")
#subset_comments_df.to_csv("comments.csv")

In [ ]:
import sys
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
#print(parent_directory)
sys.path.append(parent_directory)

from project.utils.classes.tokenizer import Tokenizer
tokenizer = Tokenizer(filepath='comments.csv', filename='tokenized_comments.csv')
tokenizer.process(cols_to_tokenize=[('body', 'tokenized_body')])

In [ ]:
tokenizer.tokenized_df

#### Method 1: Assign Moral Foundations based on Word Counts

In [ ]:
def assign_moral_foundations(comment, extended_dict):
    foundations = {}
    for word in comment:
        for key, foundation_values in extended_dict.items():
            # Ensure foundation_values is a list for consistency
            if not isinstance(foundation_values, list):
                foundation_values = [foundation_values]
            
            for value in foundation_values:
                if key.endswith('*') and word.startswith(key[:-1]):
                    # Increment the foundation's count
                    foundations[value] = foundations.get(value, 0) + 1
                elif word == key:
                    # Increment the foundation's count
                    foundations[value] = foundations.get(value, 0) + 1
    return foundations

# Assuming tokenizer.tokenized_df['tokenized_body_words_norm'] contains lists of words for each comment
tokenizer.tokenized_df['moral_foundations'] = tokenizer.tokenized_df['tokenized_body_words_norm'].apply(
    lambda x: assign_moral_foundations(x, word_to_moral_foundation_expanded)
)

In [ ]:
tokenizer.tokenized_df['moral_foundations']

#### Method 2: Using moralstrength package

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from moralstrength.moralstrength import estimate_morals, models, string_moral_values

texts = tokenizer.tokenized_df['tokenized_body_sents']

for text in texts:
    text = text
    print(text)
    #result = estimate_morals(text, process=True) # set to false if text is alredy pre-processed
    result = string_moral_values(text[0], model='unigram+freq')
    print(result)